Limitamos el análisis al día más activo de la semana.
Sólo para este día traemos embeddings y hacemos topic modeling.

Posiblemente el topic modeling requiera tomar una muestra de los tuits, para luego extenderlo al día completo.m

In [2]:
import numpy as np
import pandas as pd
import random

In [4]:
chunk_size = 10000  # Adjust the chunk size as needed
all_chunks = []

for i in range(1,8):
    chunks = pd.read_csv(f'data/tweets_batch_00{i}.tsv', sep='\t',
                         usecols=["tweet_id", "user_id", "full_text", "created_at"],
                         engine='python',
                         on_bad_lines='skip',
                         chunksize=chunk_size)
    all_chunks += chunks

df = pd.concat(all_chunks, ignore_index=True)

for f in ["user_id"]:
    df[f] = df[f].astype("Int64")

In [5]:
df_full = df

In [6]:
df_full.columns

Index(['tweet_id', 'user_id', 'created_at', 'full_text'], dtype='object')

In [7]:
df_full.shape

(6166218, 4)

In [8]:
df = df_full

In [8]:
df_full.created_at = pd.to_datetime(df_full.created_at, errors='coerce')
df_full.created_at.min(), df_full.created_at.max()

(Timestamp('2020-04-12 00:00:00'), Timestamp('2020-04-18 23:59:50'))

In [26]:
df_full["created_date"] = df_full.created_at.dt.date
day_counts = df_full.created_date.value_counts()
day_counts

created_date
2020-04-16    971679
2020-04-17    955989
2020-04-12    953722
2020-04-15    922543
2020-04-14    889848
2020-04-18    869399
2020-04-13    602529
Name: count, dtype: int64

In [15]:
# Tomamos el día más activo
day_max = day_counts.idxmax()

In [13]:
df_max_day = df_full[df_full.created_date == day_max]

In [14]:
len(df_max_day)

971679

In [16]:
df_max_day.to_csv(f"data/covid_twitter_sample_{day_max.strftime('%Y-%m-%d')}.csv", index=False)

In [18]:
docs = df_max_day.full_text.values.tolist()

In [19]:
docs[:2]

['Visitamos los hospitales de la provincia de Chimborazo, la emergencia por #Covid19 está controlada. Seguimos las directrices del presidente @Lenin Moreno de recorrer el territorio e inspeccionar las casas de salud para conocer sus realidades. \n#JuntosEcuador\n#QuédateEnCasa https://t.co/7fnk38WWiI',
 'Iker Jiménez (@navedelmisterio): "Estoy tratando el coronavirus como tratamos el Ébola, la avispa asiática, Chernóbil, Atapuerca... quien no conoce nuestra trayectoria nunca lo va a entender" https://t.co/KSh6JvbikZ https://t.co/lsrt53Y0Kf']

In [20]:
docs[0]

'Visitamos los hospitales de la provincia de Chimborazo, la emergencia por #Covid19 está controlada. Seguimos las directrices del presidente @Lenin Moreno de recorrer el territorio e inspeccionar las casas de salud para conocer sus realidades. \n#JuntosEcuador\n#QuédateEnCasa https://t.co/7fnk38WWiI'

In [21]:
from sentence_transformers import SentenceTransfo
rmer

In [22]:
# Spanish multilingual model (lightweight)
# This takes ages!
# model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
embeddings_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

In [23]:
embeddings_model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [ ]:
embeddings = embeddings_model.encode(docs, batch_size=1024, show_progress_bar=True)

Batches:   0%|          | 0/949 [00:00<?, ?it/s]

In [27]:
import numpy as np

np.save(f"data/covid_twitter_sample_{day_max.strftime('%Y-%m-%d')}.npy", embeddings)